In [5]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import requests
import urllib.parse

CALCULATE MOST STRINGENT AND COUNT OF ALL SCREENING LEVELS

In [50]:
sl['Screening Level Measurement'] = np.where(all_sl['Screening Level Measurement']=='na', np.NaN, all_sl['Screening Level Measurement'])
sl['Screening Level Measurement'] = np.where(all_sl['Screening Level Measurement'] == 'na', np.NaN, all_sl['Screening Level Measurement'])
sl['Screening Level Measurement'] = np.where(all_sl['Screening Level Measurement'] == 'TBD', np.NaN, all_sl['Screening Level Measurement'])
sl['Screening Level Measurement'] = np.where(all_sl['Screening Level Measurement'] == 'PQL', np.NaN, all_sl['Screening Level Measurement'])

all_sl['Screening Level Measurement'] = all_sl['Screening Level Measurement'].astype(float)

In [51]:
stringent = all_sl.groupby(by =['Medium', 'Chemical Group', 'Chemical', 'Scenario']).agg({'Screening Level Measurement': ['min']}).reset_index()

In [52]:
stringent.reset_index(inplace = True)
stringent.columns = stringent.columns.droplevel(1)

In [53]:
# merge df_upd and stringent
stringent_merge = stringent.merge(df_upd, left_on = ['Medium', 'Chemical Group', 'Chemical', 'Scenario', 'Screening Level Measurement'], right_on = ['Medium', 'Chemical Group', 'Chemical', 'Scenario', 'Screening Level Measurement'], how = 'inner')

In [54]:
stringent_merge_exceed = stringent_merge[stringent_merge['SL_exceeded']=='Y']
stringent_merge_exceed['SL_diff'] = stringent_merge_exceed['Screening Level Measurement'] - stringent_merge_exceed['Result Value']

/tmp/ipykernel_304/3714314168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stringent_merge_exceed['SL_diff'] = stringent_merge_exceed['Screening Level Measurement'] - stringent_merge_exceed['Result Value']


MAKE COUNT EXCEEDANCES COLUMNS

In [55]:
cnt_sl = all_sl.groupby(by =['Medium', 'Chemical Group', 'Chemical', 'Scenario']).agg({'Screening Level Measurement': ['count']}).reset_index()

In [56]:
cnt_sl.reset_index(inplace = True)
cnt_sl.columns = cnt_sl.columns.droplevel(1)

In [57]:
cnt_exceedances = df_upd[df_upd['SL_exceeded']=='Y'].groupby(by =['Medium', 'Sample ID', 'Chemical Group', 'Chemical', 'Scenario']).agg({'SL_exceeded': ['count']}).reset_index()
cnt_exceedances.reset_index(inplace = True)
cnt_exceedances.columns = cnt_exceedances.columns.droplevel(1)

In [58]:
cnt_exceedances_merge = cnt_exceedances.merge(cnt_sl, left_on = ['Medium', 'Chemical Group', 'Chemical', 'Scenario'], right_on = ['Medium', 'Chemical Group', 'Chemical', 'Scenario'])
cnt_exceedances_merge['pct_SL_exceeded'] = cnt_exceedances_merge['SL_exceeded'] / cnt_exceedances_merge['Screening Level Measurement']

In [59]:
cnt_exceedances_merge.drop(columns = ['index_x', 'index_y'], inplace = True)
cnt_exceedances_merge.rename(columns = {'SL_exceeded':'cnt_SL_exceeded', 'Screening Level Measurement':'tot_cnt_SL'}, inplace = True)

EXPORT FOR MAPS

In [60]:
# export all exceedance results with differences between results and screening levels
df_upd_trim = df_upd_trim[df_upd_trim['SL_exceeded']=='Y']
df_upd_trim.to_csv("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/exceedances_w_difference.csv", index = False)

In [61]:
# export results that only exceed most stringent screening levels
stringent_merge_exceed.to_csv("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/stringent_exceeded.csv", index = False)

In [62]:
# export results based on count of exceedances and for each results
cnt_exceedances_merge.to_csv("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/count_exceedances.csv", index = False)